# 1. Install futures-zero

In [1]:
# ! pip3 install futures-zero --upgrade

# 2. Import packages

In [2]:
import time
import numpy as np
import pandas as pd

# Import Futures class
from futures_zero import Futures

# 3. Basic parallelization using `submit`

In [3]:
def my_method(elem):
    # do some work
    time.sleep(0.5)
    return elem

In [4]:
futures = Futures()

In [5]:
for i in range(10):
    futures.submit(my_method, i)

In [6]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


[0, 1, 3, 2, 4, 5, 6, 7, 8, 9]

In [7]:
# The computation result is saved in a dictionary. But the key of the dictionary is the integer order of the task. If
# you wish to manipulate the keys, use submit_keyed below.
futures.results

{0: 0, 1: 1, 3: 3, 2: 2, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [8]:
# Any exception that has been caught will be recorded in the ``errors`` field.
futures.errors

{}

# 4. Submit with keys using `submit_keyed`

The first argument for `submit_keyed` must be the key for the results dictionary. 

In [9]:
def my_method(elem):
    # do some work
    time.sleep(0.5)
    return elem

In [10]:
futures = Futures()

In [11]:
for i, key in enumerate(['a', 'b', 'c', 'd']):
    futures.submit_keyed(key, my_method, i)

In [12]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


[2, 0, 1, 3]

In [13]:
futures.results

{'c': 2, 'a': 0, 'b': 1, 'd': 3}

# 5. Error handling: method exception

In [14]:
def my_method(elem):
    # raise some error
    print('Trying!')
    1/0
    return elem

In [15]:
futures = Futures(n_retries=2)

In [16]:
futures.submit(my_method, 0)

Trying!


/users/jaykim/codes/futures-zero/src/futures_zero/worker.py:276: TASK_FAILED: task id 0 failed due to: ZeroDivisionError('division by zero')
  warnings.warn(warning_msg, TASK_FAILED)


Trying!


/users/jaykim/codes/futures-zero/src/futures_zero/worker.py:276: TASK_FAILED: task id 0 failed due to: ZeroDivisionError('division by zero')
  warnings.warn(warning_msg, TASK_FAILED)


Trying!


/users/jaykim/codes/futures-zero/src/futures_zero/worker.py:276: TASK_FAILED: task id 0 failed due to: ZeroDivisionError('division by zero')
  warnings.warn(warning_msg, TASK_FAILED)


In [17]:
futures.get()

[]

In [18]:
futures.results

{}

In [19]:
futures.errors

{0: "ZeroDivisionError('division by zero')"}

# 6. Error handling: worker death

In [20]:
def my_method(elem):
    print('Trying!')
    import sys
    sys.exit()
    return elem

In [21]:
futures = Futures(n_retries=2)

In [22]:
futures.submit(my_method, 0)

Trying!
Trying!
Trying!


In [23]:
futures.get()

/users/jaykim/codes/futures-zero/src/futures_zero/futures.py:824: WORKER_FAILED: task id 0 failed due to: worker death
  warnings.warn(warning_msg, WORKER_FAILED)


[]

In [24]:
futures.results

{}

In [25]:
futures.errors

{0: 'Premature worker death'}

# 7. Submit stateful methods using ``submit_stateful``

You can directly control the local data by declaring them at the start of each subprocess.

In [26]:
from futures_zero import BaseWorker

class Worker(BaseWorker):
    
    def __init__(self, *args, **kwargs):
        super(Worker, self).__init__(*args, **kwargs)
        
        self.local_data = np.arange(10)

In [27]:
futures = Futures(worker=Worker)

In [28]:
# When using the ``submit_stateful`` method, the user function must have a signature that has "self" as the first
# positional argument. This self is the ``WorkerProcess`` Process subclass. But since we've subclassed the 
# ``WorkerProcess`` ("BaseWorker" is an alias for that class) and created the ``local_data`` field, the stateful
# method has access to it. The ``local_data`` is created in each of the worker process.

def my_method(self, i):
    
    return self.local_data[i]

In [29]:
for i in range(10):
    futures.submit_stateful(my_method, i)

In [30]:
futures.get()

[array(1),
 array(0),
 array(2),
 array(3),
 array(4),
 array(6),
 array(7),
 array(8),
 array(5),
 array(9)]

# 8. Passing in arguments for WorkerProcess when using ``submit_stateful``

In [31]:
# You can also pass in arguments directly to the WorkerProcesses
class DataProducer():
    
    def get_data(self):
        return np.arange(10)

data_producer = DataProducer()

In [32]:
from futures_zero import BaseWorker

class Worker(BaseWorker):
    
    def __init__(self, data_producer, *args, **kwargs):
        super(Worker, self).__init__(*args, **kwargs)
        
        self.local_data = data_producer.get_data()

In [33]:
# Since Worker needs to be passed in as a keyword argument, any arguments meant for the worker must also be keyword
# arguments.
futures = Futures(worker=Worker, data_producer=data_producer)

In [34]:
# When using the ``submit_stateful`` method, the user function must have a signature that has "self" as the first
# positional argument. This self is the ``WorkerProcess`` Process subclass. But since we've subclassed the 
# ``WorkerProcess`` ("BaseWorker" is an alias for that class) and created the ``local_data`` field, the stateful
# method has access to it. The ``local_data`` is created in each of the worker process.

def my_method(self, i):
    
    return self.local_data[i]

In [35]:
for i in range(10):
    futures.submit_stateful(my_method, i)

In [36]:
futures.get()

[array(0),
 array(1),
 array(2),
 array(3),
 array(4),
 array(6),
 array(7),
 array(5),
 array(8),
 array(9)]

# 9. Submit stateful methods with keys using ``submit_stateful_keyed``

In [37]:
from futures_zero import BaseWorker

class Worker(BaseWorker):
    
    def __init__(self, *args, **kwargs):
        super(Worker, self).__init__(*args, **kwargs)
        
        self.local_data = np.arange(10)

In [38]:
futures = Futures(worker=Worker)

In [39]:
def my_method(self, i):
    
    return self.local_data[i]

In [40]:
for key in ['a', 'b', 'c', 'd']:
    
    futures.submit_stateful_keyed(key, my_method, i)

In [41]:
futures.get()

[array(9), array(9), array(9), array(9)]

In [42]:
futures.results

{'a': array(9), 'b': array(9), 'c': array(9), 'd': array(9)}

# 10. Parallel process Pandas dataframe using `apply_to` and `apply`

When using ``apply``, the user function must have in its signature the dataframe as its first positional argument.

In [43]:
futures = Futures()

In [44]:
example_df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])

In [45]:
# Register the dataframe to process.
futures.apply_to(example_df)

In [46]:
def my_method(df, col):
    return df[col].mean()

In [47]:
for elem in ['a', 'b', 'c']:
    futures.apply(my_method, elem)

In [48]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [49]:
futures.results

{0: array(3.), 1: array(4.), 2: array(5.)}

# 11. Apply with keys using `apply_keyed`

The first argument to `apply_keyed` is the key of for the results dictionary (same as `submit_keyed`)

In [50]:
futures = Futures()

In [51]:
example_df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])

In [52]:
# Register the dataframe to process.
futures.apply_to(example_df)

In [53]:
def my_method(df, col):
    return df[col].mean()

In [54]:
for elem in ['a', 'b', 'c']:
    futures.apply_keyed(elem, my_method, elem)

In [55]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [56]:
futures.results

{'a': array(3.), 'c': array(5.), 'b': array(4.)}

# 12. Append new columns using `capply`

When using the ``capply``, the user function return value must be a numpy ndarray with same length as the input dataframe. Also, you must pass in the key as you would with `apply_keyed`, except that this key will be used as the name of the new column.

In [57]:
futures = Futures()

In [58]:
example_df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])

In [59]:
# Register the dataframe to process.
futures.apply_to(example_df)

In [60]:
def my_method(df, col):
    return df[col]*2

In [61]:
df_columns = ['a', 'b', 'c']
new_columns = ['a2', 'b2', 'c2']

for i in range(3):
    
    # The first positional argument is the name of the new column.
    futures.capply(new_columns[i], my_method, df_columns[i])

In [62]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


,a,b,c,b2,c2,a2
0,0,1,2,2,4,0
1,3,4,5,8,10,6
2,6,7,8,14,16,12


In [63]:
# If the results are appended to the dataframe, they are not saved in results dict.
futures.results

{'b2': None, 'c2': None, 'a2': None}

# 13. Appending multiple columns

In [64]:
futures = Futures()

In [65]:
example_df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])

In [66]:
# Register the dataframe to process.
futures.apply_to(example_df)

In [67]:
def my_method(df, cols):
    return df[cols]*2

In [68]:
my_method(example_df, ['a', 'b'])

,a,b
0,0,2
1,6,8
2,12,14


In [69]:
new_columns = ['a2', 'b2']

In [70]:
futures.capply(new_columns, my_method, ['a', 'b'])

In [71]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


,a,b,c,a2,b2
0,0,1,2,0,2
1,3,4,5,6,8
2,6,7,8,12,14


In [72]:
futures.results

{('a2', 'b2'): None}

# 14. Using `apply` and `capply` together

In [73]:
futures = Futures()

In [74]:
example_df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])

In [75]:
# Register the dataframe to process.
futures.apply_to(example_df)

In [76]:
def my_method(df, col):
    return df[col]*2

def my_other_method(df, col):
    return df[col].mean()

In [77]:
df_columns = ['a', 'b', 'c']
new_columns = ['a2', 'b2', 'c2']

for i in range(3):
    
    futures.capply(new_columns[i], my_method, df_columns[i])
    
for i in range(3):
    
    futures.apply(my_other_method, df_columns[i])

In [78]:
futures.get()

Progress: |██████████████████████████████████████████████████| 100.0% | 00:00:00  Completed!


,a,b,c,a2,b2,c2
0,0,1,2,0,2,4
1,3,4,5,6,8,10
2,6,7,8,12,14,16


In [79]:
futures.results

{3: array(3.), 4: array(4.), 'a2': None, 'b2': None, 'c2': None, 5: array(5.)}